# Regularización Ridge para Predicción de Humedad (_humedad)
Este notebook implementa un modelo de regresión con regularización Ridge para predecir la humedad, con todas las curvas y métricas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Cargar el dataset
dataset = pd.read_csv('../dataset/ML1_temperature.xlsx - Hoja1.csv', decimal=',')
x = dataset.drop(['recnt_Humidity','recnt_Temperature'], axis=1).values
y = dataset['recnt_Humidity'].values.reshape(-1, 1)

In [ ]:
# Separar en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento del modelo Ridge y predicción
model = Ridge(alpha=1.0, random_state=42)
model.fit(x_train, y_train.ravel())
y_val_pred = model.predict(x_val)
y_train_pred = model.predict(x_train)

## Curva de aprendizaje
Muestra cómo varía el error (MSE) al predecir humedad según el tamaño del conjunto de entrenamiento.

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(Ridge(alpha=1.0), x_train, y_train.ravel(), cv=5, scoring='neg_mean_squared_error')
plt.plot(train_sizes, -train_scores.mean(axis=1), label='Entrenamiento')
plt.plot(train_sizes, -val_scores.mean(axis=1), label='Validación')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('MSE (Humedad)')
plt.title('Curva de Aprendizaje - Ridge (Humedad)')
plt.legend()
plt.grid(True)
plt.show()

## Curva de validación
Muestra el desempeño del modelo para diferentes valores de alpha al predecir humedad.

In [ ]:
alphas = np.logspace(-4, 1, 10)
train_scores, val_scores = validation_curve(Ridge(), x_train, y_train.ravel(), param_name='alpha', param_range=alphas, cv=5, scoring='neg_mean_squared_error')

plt.figure(figsize=(8,5))
plt.semilogx(alphas, -train_scores.mean(axis=1), marker='o', color='cornflowerblue', label='Entrenamiento')
plt.semilogx(alphas, -val_scores.mean(axis=1), marker='o', color='darkorange', label='Validación')
plt.fill_between(alphas, -train_scores.mean(axis=1), -val_scores.mean(axis=1), color='gray', alpha=0.15, label='Brecha')
plt.xlabel('Alpha')
plt.ylabel('MSE (Humedad)')
plt.title('Curva de Validación - Ridge (Humedad)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Curva de paridad
Compara las predicciones de humedad con los valores reales.

In [ ]:
# Curva de paridad mejorada: Humedad Real vs Predicha (Entrenamiento y Validación)
plt.figure(figsize=(7,6))
plt.scatter(y_train, model.predict(x_train), alpha=0.7, label='Entrenamiento', color='cornflowerblue')
plt.scatter(y_val, y_val_pred, alpha=0.7, label='Validación', color='darkorange')
plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', label='Predicción perfecta')
plt.xlabel('Humedad Real')
plt.ylabel('Humedad Predicha')
plt.title(f'Humedad Real vs Predicha\nMSE Validación = {mean_squared_error(y_val, y_val_pred):.3f}')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
print(f'R^2 validación (Humedad): {r2_score(y_val, y_val_pred):.3f}')

## Curva de sesgo y varianza
Analiza la diferencia entre el error de entrenamiento y validación al predecir humedad.

In [ ]:
y_train_pred = model.predict(x_train)
print(f'MSE Entrenamiento (Humedad): {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'MSE Validación (Humedad): {mean_squared_error(y_val, y_val_pred):.3f}')
print(f'R^2 Entrenamiento (Humedad): {r2_score(y_train, y_train_pred):.3f}')
print(f'R^2 Validación (Humedad): {r2_score(y_val, y_val_pred):.3f}')

## Curva de validación cruzada
Evalúa el desempeño promedio del modelo usando validación cruzada al predecir humedad.

In [ ]:
cv_scores = cross_val_score(Ridge(alpha=1.0), x, y.ravel(), cv=5, scoring='r2')
print(f'R^2 promedio validación cruzada (Humedad): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}')

## Explicación breve
Ridge permite predecir la humedad controlando el sobreajuste. El compromiso sesgo-varianza se ajusta modificando el parámetro alpha.

In [ ]:
# Actual vs Predicho (Entrenamiento y Validación) - Ridge (Humedad)
plt.figure(figsize=(12, 6))
plt.scatter(y_train, y_train_pred, alpha=0.5, label='Entrenamiento')
plt.scatter(y_val, y_val_pred, alpha=0.5, label='Validación')
plt.xlabel('Humedad Real')
plt.ylabel('Humedad Predicha')
plt.title('Actual vs Predicho - Ridge (Humedad)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Evolución del error absoluto y resumen de métricas para Ridge (Humedad)
error_abs = np.abs(y_val.ravel() - y_val_pred.ravel())
err_mean = np.mean(error_abs)
err_max = np.max(error_abs)
err_min = np.min(error_abs)
err_std = np.std(error_abs)

plt.figure(figsize=(12,4))
plt.plot(error_abs, color='orange', label='Error absoluto')
plt.axhline(np.mean(error_abs), color='red', linestyle='--', label=f'Error promedio: {np.mean(error_abs):.4f}')
plt.xlabel('Índice de muestra (tiempo relativo)')
plt.ylabel('Error Absoluto (Humedad)')
plt.title('Evolución del error absoluto en el tiempo - Ridge (Humedad)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Histograma de la distribución de errores absolutos (Humedad)
plt.figure(figsize=(8,4))
plt.hist(error_abs, bins=20, color='royalblue', edgecolor='black', alpha=0.7)
plt.xlabel('Error Absoluto (Humedad)')
plt.ylabel('Frecuencia')
plt.title('Distribución de errores absolutos - Ridge - Humedad')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Tabla resumen de métricas para Ridge (Humedad)
from IPython.display import display, HTML
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
html_table = f'''
<table style="border-collapse:collapse; width:50%">
<tr><th style="border:1px solid black; padding:4px">Métrica</th><th style="border:1px solid black; padding:4px">Ridge</th></tr>
<tr><td style="border:1px solid black; padding:4px">MSE (Test)</td><td style="border:1px solid black; padding:4px">{mse:.4f}</td></tr>
<tr><td style="border:1px solid black; padding:4px">R² (Test)</td><td style="border:1px solid black; padding:4px">{r2:.4f}</td></tr>
<tr><td style="border:1px solid black; padding:4px">Error Promedio</td><td style="border:1px solid black; padding:4px">{err_mean:.4f}</td></tr>
<tr><td style="border:1px solid black; padding:4px">Error Máximo</td><td style="border:1px solid black; padding:4px">{err_max:.4f}</td></tr>
<tr><td style="border:1px solid black; padding:4px">Error Mínimo</td><td style="border:1px solid black; padding:4px">{err_min:.4f}</td></tr>
<tr><td style="border:1px solid black; padding:4px">Desviación Estándar del Error</td><td style="border:1px solid black; padding:4px">{err_std:.4f}</td></tr>
</table>'''
display(HTML(html_table))

## Análisis avanzado de errores y resumen de métricas
Incluye evolución del error absoluto, histograma y tabla resumen para la predicción de humedad con Ridge.